https://people.dmi.unipr.it/gianfranco.rossi/PAPERS/SC.AGP98.pdf  Narrowing the gap between Set-Constraints and
CLP(SET )-Constraints
https://dl.acm.org/doi/pdf/10.1145/365151.365169 Sets and Constraint Logic Programming 2000
https://arxiv.org/pdf/cs/0309045 A uniform approach to constraint-solving for lists,
multisets, compact lists, and sets

CSt computational set theory

What about adding term orderings to lists.
First class term orderings
t <= u in prolog.
The whole point is that they are stable under instantiation.

I guess I don't need a well founded comparison, just a ground total order?... Hmm. Well then just freeze on X < Y

https://math.stackexchange.com/questions/210711/natural-order-of-rational-trees/210730#210730 https://swi-prolog.discourse.group/t/mercios-total-order-in-prolog/9175


minikanren with first class LPO as a constraint [ X ]
But I could probably do this?

https://www.metalevel.at/prolog/sorting


Rust faster-minikanre? Lean? Some of the techniques in there sound interesting.



https://arxiv.org/abs/2001.10512 Automated Proof of Bell-LaPadula Security Properties
https://github.com/rvs314/faster-clpset-minikanren


In [ ]:
# add consstraints
# diseq constraints. two watched literal?
class State():
    vs : list[smt.ExprRef]
    goals : list[smt.BoolRef]
    subst : dict[smt.ExprRef, smt.ExprRef]
    constr : list[smt.BoolRef]


def prolog(
    vs0: list[smt.ExprRef],
    goals: list[smt.BoolRef],
    rules0: Sequence[rw.Rule | smt.QuantifierRef | smt.BoolRef],
):
    rules = [
        rule if isinstance(rule, rw.Rule) else rw.rule_of_expr(rule)
        for rule in reversed(rules0)
    ]
    todo = [(vs0, goals, {})]
    while todo:
        vs, goals, subst = todo.pop()
        if len(goals) == 0:
            yield vs, {k: t for k, t in subst.items() if any(k.eq(v) for v in vs0)}
            continue
        else:
            goal = goals.pop()
            if smt.is_true(goal):
                todo.append((vs, goals, subst))
            elif smt.is_false(goal):
                continue
            elif smt.is_and(goal):
                goals.extend(reversed(goal.children()))
                todo.append((vs, goals, subst))
            elif smt.is_or(goal):
                for child in goal.children():
                    newgoals = goals + [child]
                    todo.append((vs, newgoals, subst))
            else:
                for rule in rules:
                    rule = rule.freshen()
                    vs1 = vs + rule.vs
                    subst1 = kd.utils.unify(vs1, rule.conc, goal)
                    if subst1 is None:
                        continue
                    else:
                        newgoals = goals + [smt.substitute(rule.hyp, *subst1.items())]
                        newsubst = {
                            **{
                                k: smt.substitute(v, *subst1.items())
                                for k, v in subst.items()
                            },
                            **subst1,
                        }
                        newvs = list(set(vs1) - set(subst1.keys()))
                        todo.append((newvs, newgoals, newsubst))

# 09/24

Nada has a clpset. setlog. Currently maintained? That ASP setlog idea.
Unordered sets in haskell. Observations / separation queries
<https://byorgey.github.io/blog/posts/2024/06/25/unordered-n-tuple-product.html>
https://danryba253.github.io/danirybeblog/posts/unordered-n-tuples/
https://gelisam.blogspot.com/2013/07/the-commutative-monad.html

Unification is condicutive,. cogruence closure is inductive.

Sets are coinductive.
elem : S -> enum A analog of a NFA terasnition
obs : Pred -> Either Bool (S,S)

Compare to considerations of non-well founded sets aczel. Kind of similar.

Sets are opaque. Material sets vs structural sets. ETCS. 
https://ncatlab.org/nlab/show/SEAR
https://ncatlab.org/nlab/show/structural+set+theory
https://ncatlab.org/nlab/show/material+set+theory
https://golem.ph.utexas.edu/category/2013/01/from_set_theory_to_type_theory.html Shulman

Predicates asre conductive

https://www.clpset.unipr.it/setlog.Home.html
https://www.clpset.unipr.it/SETLOG/calculus.pdf rewrite rules for set solver



https://github.com/namin/clpset-miniKanren
https://github.com/namin/clpsmt-miniKanren/blob/master/clpset.scm

https://dl.acm.org/doi/10.1145/365151.365169 Sets and constraint logic programming




<https://www.clpset.unipr.it/>
`{log}` "setlog"
JSetL

[set unification](https://www.cambridge.org/core/journals/theory-and-practice-of-logic-programming/article/abs/set-unification/2B789EB32DCF9F77021DBE26FC691032)
[Sets and constraint logic programming](https://dl.acm.org/doi/10.1145/365151.365169)

G Rossi
A Dovier
E Pontelli

herditraryl hybrid finite sets. Finite sets of finite sets + terms

[Set Constraints in Logic Programming](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=ea756dcfc1db14307899a5b7d33b82fefbdc74c0)
[Set constraints and logic programming - kozen](https://www.cs.cornell.edu/~kozen/Papers/sclp.pdf) CLP(SC)
Herbrand atom ~ singleton set
Aiken

Relation to boolean equation systems?

Kuper - Logic programming with sets

https://users.dimi.uniud.it/~agostino.dovier/DID/LNC/CONVEGNI/DOVIER_TORINO.pdf Logic Programming and Bisimulation
accesible pointed graph. "Set graphs"
https://ceur-ws.org/Vol-1433/tc_10.pdf  Logic Programming and Bisimulation∗

https://www.sciencedirect.com/science/article/pii/S0304397514004411  Set graphs. II. Complexity of set graph recognition and similar problems

mostowski's collapsing lemma
referee aetnova set proof checker
SETL

https://github.com/lparolari/setlog-picat 
https://github.com/lparolari/bachelor-thesis uh oh its in italian
https://github.com/lparolari/bachelor-thesis-presentation

https://plato.stanford.edu/entries/nonwellfounded-set-theory/
closed sets vs open sets. Closed sets have no more observations possible.
X = [A,B,C,D]
When do we close off sets?
Somehow rational terms come more naturally to prolog. Occurs check is something we add on top. Unification variables are lazy? Somehow they are more condinductive of something. Observation like qauntum observation. Forcing. Querying.
Logic variables are condiuctive entitities?
Observational entities. Defined by elimination rules rather than introduction rules.


Unfounded sets as prolog semantics. If we don't require a straightening out into normal finite sets and allow loops, does prolog have an easier smenatics with regards to negation? https://arxiv.org/abs/1401.4596  Unfounded Sets and Well-Founded Semantics of Answer Set Programs with Aggregates
https://dl.acm.org/doi/abs/10.1145/308386.308444 Unfounded Sets and Well-Founded Semantics for General Logic Programs  van gelder
No, I don't think they are connecting "unfounded" to "non-well-founded"

Remmber thatere was that idea that :- is element of and that d :- {a,b,c} means {a,b,c} \elem d. Interpeation as justification sets.

In [ ]:
# knuckledragger style
# https://plato.stanford.edu/entries/nonwellfounded-set-theory/#4.1
class ClassCat():
    x
    y
    dom: BoolRef
    cod: BoolRef
    


union inherits the observations of it's pieces
inter inherits the equal observations of it's pieces

In [ ]:
# finite set theory style

# automata minimization is the analog of hash consing.
# hash consing abuses the fact that we build things in a well founded way anyhow, that may be why it is cleaner

# The "close" interface is another variation on 

class Univ():
    counter = 0
    obs : dict[setid, ]
    closed : set[setid]
    empty = 0
    def makeset(self):
        self.counter += 1
        return self.counter
    def close(self, setid):
        self.closed.add(setid)
    def minimize(self):
    #def union(self, setid, setid): # hmm.
    #    pass
    def part(self, setid, pred, setid):
        # partition / separate the set.
        # part(a,p,b) says that b is the part of a that (provably?) obeys p. we're a bit agnostic if p is decidable / LEM?
        # we are agnostic about whether the set is finite or not kind of.
    #def elem(elem, setid, setid):
    # pass maybe this is jusrt a particular predicate


In [ ]:
from prologprovers import setlog
import janus_swi as janus

janus.consult(setlog.path)